In [1]:
import os, pickle, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
for p in [np, pd]:
    print (p.__name__, p.__version__)

numpy 1.18.1
pandas 1.0.1


In this file, we load test data and previously trained model, generate prediction, and save them for submission.

# Load Test Data and Trained Models

In [3]:
test_df = pd.read_hdf('../CleanData/testDF.h5', 'df')

In [4]:
filename = "../Model/lgbm_model_full.pkl"
with open(filename, 'rb') as file:
    lgbm_model_full = pickle.load(file)

In [5]:
filename = "../Model/lr_model_full.pkl"
with open(filename, 'rb') as file:
    lr_model_full = pickle.load(file)

# Generate Predictions for All Trained Models

In [6]:
lgbm_features = ['avg_item_price_lag_1', 'target_lag_1', 'target_shop_lag_1', 'target_item_lag_1', 
                 'target_item_category_lag_1', 'avg_item_price_lag_2', 'target_lag_2', 'target_shop_lag_2',
                 'target_item_lag_2', 'target_item_category_lag_2', 'avg_item_price_lag_3', 'target_lag_3', 
                 'target_shop_lag_3', 'target_item_lag_3', 'target_item_category_lag_3', 'avg_item_price_lag_4', 
                 'target_lag_4', 'target_shop_lag_4', 'target_item_lag_4', 'target_item_category_lag_4',
                 'avg_item_price_lag_5', 'target_lag_5', 'target_shop_lag_5', 'target_item_lag_5', 
                 'target_item_category_lag_5', 'avg_item_price_lag_6', 'target_lag_6', 'target_shop_lag_6',
                 'target_item_lag_6', 'target_item_category_lag_6', 'avg_item_price_lag_12', 'target_lag_12', 
                 'target_shop_lag_12', 'target_item_lag_12', 'target_item_category_lag_12', 'shop_mean',
                 'item_mean', 'shop_item_mean', 'item_category_mean', 'month']
Z_test_lgbm_full = lgbm_model_full.predict(test_df[lgbm_features]).clip(0,20)

In [7]:
lr_features = ['target_lag_1', 'target_lag_2', 'target_lag_3', 'target_lag_4', 'target_lag_5', 'target_lag_6', 
               'shop_mean', 'item_mean', 'shop_item_mean', 'item_category_mean']
Z_test_lr_full = lr_model_full.predict(test_df[lr_features]).clip(0,20)

# Ensemble Model Predictions

Finally, we will do a simple convex combination of light GBM and linear model predictions.  The best alpha was found in train.py on validation data.

In [8]:
best_alpha = 0.974
Z_test_mix = best_alpha*Z_test_lgbm_full + (1 - best_alpha)*Z_test_lr_full

# Create submission

In [9]:
ensemble_submission = pd.DataFrame(Z_test_mix).reset_index()
ensemble_submission.columns = ['ID', 'item_cnt_month']
ensemble_submission.to_csv('../Submission/ensemble_submission.csv', index=False) 